In [74]:
from random import uniform
from math import cos, sin, pi
import matplotlib.pyplot as plt
import random
import numpy as np
from sklearn import svm

In [31]:
import snoop

In [115]:
def generate1():  
    a = uniform(0, 1)  
    b = uniform(0, 1)  
    return a * cos(2 * pi * b), a * sin(2 * pi * b)

def generate2():  
    while True:  
        x = uniform(-1, 1)  
        y = uniform(-1, 1)  
        if x ** 2 + y ** 2 > 1:  
            continue  
        return x, y

In [96]:
def generate_train_dataset(n_samples):
    labels = [random.choice([1, 2]) for i in range(n_samples)]
    dataset = []
    for i in range(n_samples):
        if labels[i] == 1:
            dataset.append([generate1() for i in range(1000)])
        elif labels[i] == 2:
            dataset.append([generate2() for i in range(1000)])
    return np.array(dataset).sum(axis=-1), labels

In [126]:
dataset, labels = generate_train_dataset(10_000);

In [98]:
np.shape(dataset)

(10000, 1000)

In [41]:
#@snoop
def kNN(dataset, circle, k, labels):
    diffmat = np.tile(circle, (np.shape(dataset)[0], 1, 1)) - dataset
    distances = (((diffmat ** 2).sum(axis=2))**0.5).sum(axis=1)
    sorted_dist_ind = distances.argsort()
    class_count = {}
    for i in range(k):
        vote_label = labels[sorted_dist_ind[i]]
        class_count[vote_label] = class_count.get(vote_label, 0) + 1
    return class_count

In [125]:
clf = svm.SVC()
clf.fit(dataset, labels);

In [123]:
'''test, labels_test = generate_train_dataset(100)
res = [clf.predict(np.reshape(test[i], (1,1000))) == labels_test[i] for i in range(100)]'''

In [130]:
def define_size():
    test, labels_test = generate_train_dataset(100)
    error = []
    for i in range(1, 5):
        dataset, labels = generate_train_dataset(10**i)
        clf = svm.SVC()
        clf.fit(dataset, labels)
        res = [clf.predict(np.reshape(test[i], (1,1000))) == labels_test[i] for i in range(100)]
        error.append(res.count(True)/len(res))
    return error

In [129]:
np.shape(dataset)

(10000, 1000)

In [131]:
define_size()

[0.84, 1.0, 1.0, 1.0]

Попробовать алгоритмы:
1. Персептрон Розенблатта
2. (Собственный код для классификатора на опорных векторах)
3. Оптимальная разделяющая гиперплоскость
4. Линейный дискриминантный анализ?

Необходимо решить задачу выпуклой оптимизации:
$$
\left\{\begin{array}{l}
\frac{1}{2}\|w\|^{2}+C \sum_{i=1}^{\ell} \xi_{i} \rightarrow \min _{w, b, \xi} \\
y_{i}\left(\left\langle w, x_{i}\right\rangle+b\right) \geqslant 1-\xi_{i}, \quad i=1, \ldots, \ell \\
\xi_{i} \geqslant 0, \quad i=1, \ldots, \ell
\end{array}\right.
$$